# EEG Emotion detection from alpha and beta band entropies on simplified dataset


In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import confusion_matrix, classification_report

In [5]:
# Here is how we extract the features and the labels from the original 55 MB dataset
import csv
emotion_data = pd.read_csv("emotions.csv")   
emotion_data.info()
# give the head only for labels
emotion_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2132 entries, 0 to 2131
Columns: 2549 entries, # mean_0_a to label
dtypes: float64(2548), object(1)
memory usage: 41.5+ MB


,# mean_0_a,mean_1_a,mean_2_a,mean_3_a,mean_4_a,mean_d_0_a,mean_d_1_a,mean_d_2_a,mean_d_3_a,mean_d_4_a,...,fft_741_b,fft_742_b,fft_743_b,fft_744_b,fft_745_b,fft_746_b,fft_747_b,fft_748_b,fft_749_b,label
0,4.62,30.3,-356.0,15.6,26.3,1.070,0.411,-15.70,2.06,3.15,...,23.5,20.3,20.3,23.5,-215.0,280.00,-162.00,-162.00,280.00,NEGATIVE
1,28.80,33.1,32.0,25.8,22.8,6.550,1.680,2.88,3.83,-4.82,...,-23.3,-21.8,-21.8,-23.3,182.0,2.57,-31.60,-31.60,2.57,NEUTRAL
2,8.90,29.4,-416.0,16.7,23.7,79.900,3.360,90.20,89.90,2.03,...,462.0,-233.0,-233.0,462.0,-267.0,281.00,-148.00,-148.00,281.00,POSITIVE
3,14.90,31.6,-143.0,19.8,24.3,-0.584,-0.284,8.82,2.30,-1.97,...,299.0,-243.0,-243.0,299.0,132.0,-12.40,9.53,9.53,-12.40,POSITIVE
4,28.30,31.3,45.2,27.3,24.5,34.800,-5.790,3.06,41.40,5.52,...,12.0,38.1,38.1,12.0,119.0,-17.60,23.90,23.90,-17.60,NEUTRAL


# Load the data

In [6]:
# LABELS to classes mapping
maps = dict({"NEGATIVE":2,"POSITIVE":1,"NEUTRAL":0})


# Split data into training and testing

In [7]:
from sklearn.preprocessing import OneHotEncoder
# We want to predict the emotion based on the entropy features
y = pd.get_dummies(emotion_data['label'].apply(lambda x: maps[x]))
# X is everything else but the label
X = emotion_data.drop('label', axis=1).copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

In [8]:
import pandas as pd
from sklearn import preprocessing

x = X_train #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)

# Neural Network

In [9]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import keras


model = models.Sequential()
model.add(tf.keras.Input(shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation="softmax"))

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │       652,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 685,827 (2.62 MB)

 Trainable params: 685,827 (2.62 MB)

 Non-trainable params: 0 (0.00 B)

None


In [10]:
model.fit(df,y_train,epochs=30,batch_size=32)

Epoch 1/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4265 - loss: 1.3081
Epoch 2/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7750 - loss: 0.6483
Epoch 3/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8057 - loss: 0.5079
Epoch 4/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8711 - loss: 0.3905
Epoch 5/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8359 - loss: 0.4249
Epoch 6/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8858 - loss: 0.3370
Epoch 7/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8830 - loss: 0.3146
Epoch 8/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8644 - loss: 0.3429
Epoch 9/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8929 - loss: 0.3035
Epoch 10/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8816 - loss: 0.3008
Epoch 11/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8867 - loss: 0.2925
Epoch 12/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8907 - lo

In [11]:
import pandas as pd
from sklearn import preprocessing

x = X_test #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)

model.evaluate(df,y_test)

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 980us/step - accuracy: 0.4606 - loss: 7.1584


[7.084458827972412, 0.4593749940395355]

In [20]:
# CNN, with first conv1d layer 16x10, second 16x3 and maxpooling 8, with dense1 100 and dense2 3
model = models.Sequential()
model.add(tf.keras.Input(shape=(x_scaled.shape[1],)))
model.add(tf.keras.layers.Reshape((x_scaled.shape[1],1)))
model.add(tf.keras.layers.Conv1D(16, 10, activation='relu'))
model.add(tf.keras.layers.Conv1D(16, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(8))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation="softmax"))
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_2 (Reshape)             │ (None, 2548, 1)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_7 (Conv1D)               │ (None, 2539, 16)       │           176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_8 (Conv1D)               │ (None, 2537, 16)       │           784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 317, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 5072)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 100)            │       507,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 3)              │           303 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 508,563 (1.94 MB)

 Trainable params: 508,563 (1.94 MB)

 Non-trainable params: 0 (0.00 B)

In [33]:
# Convert y_train and y_test back to integer labels
y_train_int = np.argmax(y_train.values, axis=1)
y_test_int = np.argmax(y_test.values, axis=1)

# Rescale X_train to match the input shape expected by the model
x_train = X_train # returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_train_scaled = min_max_scaler.fit_transform(x_train)
df_train = pd.DataFrame(x_train_scaled)

# Reshape X_train to match the input shape expected by the model
X_train_reshaped = df_train.values.reshape((df_train.shape[0], df_train.shape[1], 1))
# set adam learning rate of 0.0001 and decayof 1e−6
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train_reshaped, y_train_int, epochs=100, batch_size=16)

Epoch 1/100


/opt/homebrew/Caskroom/miniforge/base/envs/TF211/lib/python3.10/site-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9320 - loss: 0.1572
Epoch 2/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9385 - loss: 0.1534
Epoch 3/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9422 - loss: 0.1557
Epoch 4/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9465 - loss: 0.1488
Epoch 5/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9563 - loss: 0.1411
Epoch 6/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9504 - loss: 0.1371
Epoch 7/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9406 - loss: 0.1526
Epoch 8/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9376 - loss: 0.1732
Epoch 9/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9498 - loss: 0.1460
Epoch 10/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9438 - loss: 0.1547
Epoch 11/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9531 - loss: 0.1308
Epoch 12/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9409 - lo

In [34]:
model.evaluate(df,y_test_int)

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5373 - loss: 10.5989


[10.294724464416504, 0.5453125238418579]

In [36]:
# confusion matrix
y_pred = np.argmax(model.predict(df), axis=-1)
confusion_matrix(y_test_int, y_pred)


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


array([[225,   0,   0],
       [185,   7,   6],
       [ 82,  18, 117]])